In [1]:
!pip install -q transformers sentence-transformers spacy protobuf==3.20.3
!pip install -q torch torchvision torchaudio
!pip install -q torch-scatter torch-geometric

In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
pairs_df = pd.read_csv("/kaggle/input/datasets/mahbubahabib/normalized-openqa/pairs_df_with_normalized_edges.csv")
conceptnet_df = pd.read_csv("/kaggle/input/datasets/mahbubahabib/trycandy/conceptnet.en (1).csv", sep="\t")
edge_embeddings = np.load("/kaggle/input/datasets/mahbubahabib/edge-embedding/conceptnet_edge_embeddings (1).npy")

#MODEL INTIALIZATION AND MODEL EVAL() SILO


In [11]:
def edge_id_to_tuple(edge_id, weight):
    row = conceptnet_df.iloc[int(edge_id)]
    return row["head"], row["relation"], row["tail"], weight

In [36]:
conceptnet_df = conceptnet_df.rename(columns={
    "antonym": "relation",
    "ab_extra": "head",
    "ab_intra": "tail",
    "1.0": "weight"
})
conceptnet_df

,relation,head,tail,weight
0,antonym,ab_intra,ab_extra,1.0
1,antonym,abactinal,actinal,1.0
2,antonym,abandon,acquire,1.0
3,antonym,abandon,arrogate,1.0
4,antonym,abandon,embrace,1.0
...,...,...,...,...
2487804,usedfor,zoom_lens,procure_better_shot,1.0
2487805,usedfor,zoom_lens,see_things_bigger,1.0
2487806,usedfor,zoom_lens,seeing_distant_object_more_closely,1.0
2487807,usedfor,zoom_lens,take_pictures,1.0


In [37]:


import numpy as np
import ast

def safe_parse_edges(x):
    # if string from CSV → convert to list
    if isinstance(x, str):
        x = ast.literal_eval(x)
    return x
def softmax_normalize_edges(edge_list):
    if not edge_list:
        return []

    cleaned = []

    for e in edge_list:
        # case: (id, weight)
        if isinstance(e, (list, tuple)) and len(e) >= 2:
            eid = int(e[0])
            w = float(e[1])
            cleaned.append((eid, w))

        # case: (id,) or just id
        elif isinstance(e, (list, tuple)) and len(e) == 1:
            cleaned.append((int(e[0]), 0.0))

        elif isinstance(e, (int, np.integer)):
            cleaned.append((int(e), 0.0))
        else:
            continue

    if not cleaned:
        return []

    scores = np.array([w for (_, w) in cleaned], dtype=float)

    exp_scores = np.exp(scores)
    weights = exp_scores / exp_scores.sum()

    return [(cleaned[i][0], float(weights[i])) for i in range(len(cleaned))]


pairs_df["graph_edges"] = pairs_df["graph_edges"].apply(safe_parse_edges)
pairs_df["graph_edges_norm"] = pairs_df["graph_edges"].apply(softmax_normalize_edges)

In [38]:

def concept_overlap(sentence, question):
    s_words = set(sentence.lower().split())
    q_words = set(question.lower().split())
    return len(s_words & q_words)

In [28]:
def edges_to_sentences(edge_list):
    sentences = []
    for e in edge_list:
        # unpack safely
        edge_id = e[0]
        weight = e[1] if len(e) > 1 else 1.0  # default weight 1.0 if missing

        h, r, t, w = edge_id_to_tuple(edge_id, weight)
        sent = f"{h} {r} {t}. (score={w:.2f})"
        sentences.append((sent, w))
    return sentences

In [39]:
def get_top_k_sentences(edge_list, k=20):
    sents = edges_to_sentences(edge_list)
    sents = sorted(sents, key=lambda x: x[1], reverse=True)  # by weight
    return sents[:k]

In [40]:
def llm_score(question, answer, fact_sentence):
    prompt = f"""
    
Question: {question}
Answer: {answer}

Fact: {fact_sentence}

Does the fact support the answer to the question?
Reply only with a number between 0 and 1.
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=10,
            do_sample=False
        )

    text = tokenizer.decode(output[0], skip_special_tokens=True)

    # extract float
    import re
    nums = re.findall(r"0\.\d+|1\.0|1", text)
    if len(nums) == 0:
        return 0.0
    return float(nums[-1])

In [41]:
def process_row(row):
    edge_list = row["graph_edges"]

    top_sentences = get_top_k_sentences(edge_list, k=20)

    scores = []
    for sent, w in top_sentences:
        s = llm_score(row["question"], row["answer"], sent)
        scores.append(s)

    return max(scores)   # best supporting fact

In [42]:
print(pairs_df["graph_edges"].iloc[0][:5])
edges_to_sentences(pairs_df["graph_edges"].iloc[0][:5])

[(40513, -3.6227420637112284), (2419135, -5.1974027893417185), (1989233, -5.1974027893417185), (2419131, -5.1974027893417185), (2482166, -5.322172369307295)]


[('small_dog atlocation sweden. (score=-3.62)', -3.6227420637112284),
 ('sun_bear relatedto sun_bear. (score=-5.20)', -5.1974027893417185),
 ('sun relatedto sunn. (score=-5.20)', -5.1974027893417185),
 ('sun relatedto sun. (score=-5.20)', -5.1974027893417185),
 ('sun usedfor shining_during_day. (score=-5.32)', -5.322172369307295)]

In [43]:
pairs_df["llm_score"] = pairs_df.apply(process_row, axis=1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [ ]:
pairs_df_small = pairs_df.head(4)
pairs_df_small["llm_score"] = pairs_df_small.apply(process_row, axis=1)
pairs_df_small

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
pairs_df["pred_label"] = 0

for qid, group in pairs_df.groupby("qid"):
    best_idx = group["llm_score"].idxmax()
    pairs_df.loc[best_idx, "pred_label"] = 1

In [ ]:
pairs_df.to_csv("blessings_with_llm_predictions.csv", index=False)